In [1]:
import time
import serial
import numpy as np
import sys
import os
import asyncio
from multiprocessing import Process
import threading
from scipy import constants
import itertools

import binascii
from datetime import datetime
import logging

#from gsioc import gsioc_Protocol, ensure_xy_position_will_be_reached

In [2]:
from gilson import *

In [3]:
from platform_setup_new import *

In [4]:
from hb_elite_pump.hb_elite import *

# Connecting to Equipment

To reset the COM port:
1) Open device manager and disable COM1
2) Turn machine off and on
3) Enable COM1 in device manager

Use ser.close() here to close the serial port

NOTE: Python seems to have problems when the GX-241 is in COM1 (motherboard), it will connect but cannot send or recieve. This is easily fixed by using a USB to serial adapter instead.

### Autosampler

In [5]:
# Set up the serial connection
PORT1 = 'COM6'
ser = serial.Serial(PORT1, 19200, 8, "N", 1, 0.1)

In [6]:
# Create an instance of the gsioc_Protocol class
g = gsioc_Protocol(ser, 'GX-241 II', 30)

In [7]:
g.enable_logging()

Logging enabled.


In [8]:
g.log_info('Test log.')

In [9]:
g.connect()

Connected to autosampler


In [10]:
g.get_device_name()

'GX-241 II v2.0.2.5'

In [11]:
g.iCommand('e')

'0 No Error'

In [12]:
g.bCommand('H')

bytearray(b'\nH\r')

In [16]:
g.go_to_vial(1)

In [ ]:
# Create an instance of the gsioc_Protocol class
g_dim = gsioc_Protocol(ser, 'GX D Inject', 3)

In [ ]:
g_dim.connect()

In [ ]:
g_dim.get_device_name()

### SyrPump

In [13]:
#Defining ser_1
PORT2 = 'COM4'
ser_1 = serial.Serial(PORT2, 9600, 8, "N", 1, 0.1)

In [14]:
# Create an instance of the gsioc_Protocol class
hbpump = HBElite(ser_1)

In [15]:
#Connecting to pump
hbpump.connect()

Device is connected


### Knauer Pump

In [ ]:
#Defining ser_2
PORT3 = 'COM7'
ser_2 = serial.Serial(PORT3, 9600, 8, "N", 1, 0.1)

In [ ]:
# Create an instance of the gsioc_Protocol class
kpump = K100Pump(ser_1)

In [ ]:
kpump.connect()

## Try Run and Platform Setup

This seems a little complex as is, too many unneeded things bundled in run.py. Try to extract the relevant functions and start again.

In [16]:
rack_position_offset_x=92       #distance in mm between rack_position=1 and =2 on x-axis
rack_position_offset_y=0        #distance in mm between rack_position=1 and =2 on y-axis

############################# RACK 1 DEFINITION #################################

# From platform_setup.py 
rack1 = Rack([4,16], 7.5, 39.5, 18.5, 13.75, 65) # groundlevel_height assumed the minimum Z

#  array_dimensions, offset_x, offset_y=offset_y, vial2vial_x, vial2vial_y, groundlevel_height

# Previous vial2vialx = (2.11+15.6)
# Previous vial2vial7 = (2.72+15.6+0.35)

array_order1 = np.array([      #user is obliged to define a integer number i>=1 for each vial in the rack in ascending order 
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10,11,12],
    [13,14,15,16],
    [17,18,19,20],
    [21,22,23,24],     
    [25,26,27,28],     
    [29,30,31,32],
    [33,34,35,36],
    [37,38,39,40],
    [41,42,43,44],
    [45,46,47,48],
    [49,50,51,52],
    [53,54,55,56],
    [57,58,59,60],
    [61,62,63,64]        
    ])
    
rack_pos1=1

global rack1_commands

# Not sure what rack_position_offset_x/y are for x=92 and y=0

rack1_commands = Rackcommands(rack1, array_order1, rack_pos1, rack_position_offset_x, rack_position_offset_y)

global vial_selfmade

vial_selfmade = Vial(1.5, 1, 33, 31.08)

In [17]:
# It works!
thing = rack1_commands.get_xy_command(3)
thing
g.bCommand(thing[0])

bytearray(b'\nX44.5/39.5\r')

In [8]:
g.bCommand('H')

bytearray(b'\nH\r')

## With syr pump

In [33]:


hbpump.command('cvolume')

hbpump.command('stop')


['', ':']

In [15]:
g.go_to_home()

# Preparing Reaction Plugs

In [43]:
# Define kinetic screen
no_sub_A = 2      # First Column
no_sub_B = 2      # Second Column
no_cat = 2        # Third Column
no_add = 2        # Fourth Column

# Create lists for each
sub_A_list = [1 + 4 * i for i in range(no_sub_A)]
sub_B_list = [2 + 4 * i for i in range(no_sub_B)] if no_sub_B > 0 else []
cat_list = [3 + 4 * i for i in range(no_cat)] if no_cat > 0 else []
add_list = [4 + 4 * i for i in range(no_add)] if no_add > 0 else []

#print(sub_A_array, sub_B_array, cat_array)

In [44]:
# Generate all possible combinations
all_combinations = list(itertools.product(sub_A_list, sub_B_list, cat_list, add_list))

# Reorder combinations so that all combinations with the same elements
# in sub_B_array and cat_array are grouped together
spka_combinations = []

# Iterate through all unique combinations of sub_B_array and cat_array
# This will go through all sub_A first, then sub_B, then cat
for a in sub_A_list:
    if sub_B_list:
        for b in sub_B_list:
            if cat_list:  # Check if cat_list is not empty
                for c in cat_list:
                    if add_list:  # Only include d if add_list is not empty
                        for d in add_list:
                            spka_combinations.append((a, b, c, d))
                    else:
                        spka_combinations.append((a, b, c))  # Append without d
            else:  # If cat_list is empty
                if add_list:
                    for d in add_list:
                        spka_combinations.append((a, b, d))  # Append without c
                else:
                    spka_combinations.append((a, b))  # Append only a and b
    else:  # If sub_B_list is empty
        if cat_list:
            for c in cat_list:
                if add_list:
                    for d in add_list:
                        spka_combinations.append((a, c, d))  # Append without b
                else:
                    spka_combinations.append((a, c))  # Append only a and c
        else:  # If both sub_B_list and cat_list are empty
            if add_list:
                for d in add_list:
                    spka_combinations.append((a, d))  # Append only a and d
            else:
                spka_combinations.append((a,))  # Append only a

# Display the combinations
spka_combinations

[(1, 2, 3, 4),
 (1, 2, 3, 8),
 (1, 2, 7, 4),
 (1, 2, 7, 8),
 (1, 6, 3, 4),
 (1, 6, 3, 8),
 (1, 6, 7, 4),
 (1, 6, 7, 8),
 (5, 2, 3, 4),
 (5, 2, 3, 8),
 (5, 2, 7, 4),
 (5, 2, 7, 8),
 (5, 6, 3, 4),
 (5, 6, 3, 8),
 (5, 6, 7, 4),
 (5, 6, 7, 8)]

In [45]:
# Iterate over each tuple in the list (each separate kinetic profile)
for profile in spka_combinations:
    
    # Iterate over each number in the tuple (each reagent in an experiment)
    for vial in profile:
        
        # Display vial number - must be a two digit integer
        g.bCommand(f"W{vial:02}")
        
        # Go to Vial
        g.go_to_vial(vial)
        log_action('device_log.txt', f"Autosampler sent to position {vial:02}")
        
        # Needle Down - define this distance somewhere above!
        g.bCommand('Z85')

        #HBElite to withdraw
        pump.command('cvolume')
        pump.command('ctvolume')
        pump.command(f'tvolume {vol} ul') # need to work out somewhere a set volume
        pump.command('wrate 100 ul/min')
        pump.command('wrun')
        time.sleep(vol / 100)
        #need to put a sleep to make code wait for withdrawal before next step
        pump.command('stop')

        
        # Run the pump
        # This will be a while away...
        # How to make the autosampler idle while this is happening?

        # Needle Up
        g.bCommand('Z120')

        # Log the vial and the amount taken
        log_action('device_log.txt', f"{vol} taken from position {vial:02}")

        # Create air gap
        pump.command('cvolume')
        pump.command('ctvolume')
        pump.command('tvolume 2 ul') # need to work out somewhere a set volume
        pump.command('wrate 40 ul/min')
        pump.command('wrun')
        time.sleep(3)
        
        # Create a tiny airgap with the pump, will this be necessary?

    
    # Go to the DIM and inject
    g_inject()
    log_action('device_log.txt', "Autosampler sent to DIM to inject")
    
    # Blank the display
    g.bCommand('WBB')

log_action('device_log.txt', "Reaction plug preparation complete")

g.bCommand('H')
log_action('device_log.txt', "Autosampler sent to home position")

NameError: name 'pump' is not defined

# Syringe Pump Validation

In [1]:
#need to make up solution of x concentration of BnOH in MeCN

### taking set amounts

In [ ]:
#for loop probably the ebst way to go

volumes_1 = {20:1, 40:2, 60:3, 80:4, 100:5, 120:6}
#max flow rate is 132.6 ul/min for 50 uL syringe
#max flow rate is  381.9 ul/min for 100 ul syringe
#for 1000ul syringe max flow is 2.654 ml/min

hbpump.command('irate 1000 ul/min')#
hbpump.command('wrate 200 ul/min')

for volume, vial in volumes_1.items():
    #go to the stock sol vial
    g.go_to_vial(8)
    print('At stock solution vial.')
    g.bCommand('Z85')
    
    #syr pump withdraw
    hbpump.set_tvolume(f'{volume} ul') # need to work out somewhere a set volume
    hbpump.withdraw()
    time.sleep(60 * (volume / 200))
    #need to put a sleep to make code wait for withdrawal before next step
    hbpump.clear_volume()

    #for needle to go up
    g.bCommand('Z125')
    
    #go to vial
    g.go_to_vial(vial)
    g.bCommand('Z85')
    print(f'At vial {vial}.')

    #syr pump infuse
    hbpump.set_tvolume(f'{volume} ul') # need to work out somewhere a set volume
    hbpump.infuse()
    print(f'Infusing {volume} uL')
    time.sleep(60 * (volume / 1000))
    #need to put a sleep to make code wait for withdrawal before next step
    hbpump.clear_volume()

    #for needle to go up down up
    g.bCommand('Z125')
    g.bCommand('Z85')
    g.bCommand('Z125')

    #a blank vial for to clean the needle
    g.go_to_vial(7)
    g.bCommand('Z85')
    g.bCommand('Z125')
    g.bCommand('Z85')
    g.bCommand('Z125')
    
g.go_to_home()


In [36]:
#for loop probably the ebst way to go

volumes_1 = {10:1, 25:2, 50:3, 100:4}
#max flow rate is 132.6 ul/min for 50 uL syringe
#max flow rate is  381.9 ul/min for 100 ul syringe
#for 1000ul syringe max flow is 2.654 ml/min

hbpump.command('irate 100 ul/min')
hbpump.command('wrate 300 ul/min')

irate = 100
wrate = 300

for volume, vial in volumes_1.items():
    #go to the stock sol vial
    g.go_to_vial(8)
    g.bCommand('Z85')
    
    #syr pump withdraw
    hbpump.set_tvolume(f'{volume} ul') # need to work out somewhere a set volume
    hbpump.withdraw()
    time.sleep(10)
    time.sleep(60 * (volume / wrate))
    #need to put a sleep to make code wait for withdrawal before next step
    hbpump.clear_volume()

    #for needle to go up
    g.bCommand('Z125')
    
    #go to vial
    g.go_to_vial(vial)
    g.bCommand('Z85')
    print(f'At vial {vial}.')

    #syr pump infuse
    hbpump.set_tvolume(f'{volume} ul') # need to work out somewhere a set volume
    hbpump.infuse()
    time.sleep(5)
    time.sleep(60 * (volume / irate))
    #need to put a sleep to make code wait for withdrawal before next step
    hbpump.clear_volume()

    #for needle to go up down up
    g.bCommand('Z125')
    g.bCommand('Z85')
    g.bCommand('Z125')

    
g.go_to_home()


volumes_2 = {10:9, 25:10, 50:11, 100:12}

for volume, vial in volumes_2.items():
    #go to the stock sol vial
    g.go_to_vial(16)
    g.bCommand('Z85')
    
    #syr pump withdraw
    hbpump.set_tvolume(f'{volume} ul') # need to work out somewhere a set volume
    hbpump.withdraw()
    time.sleep(2)
    time.sleep(60 * (volume / wrate))
    #need to put a sleep to make code wait for withdrawal before next step
    hbpump.clear_volume()

    #for needle to go up
    g.bCommand('Z125')
    
    #go to vial
    g.go_to_vial(vial)
    g.bCommand('Z85')
    print(f'At vial {vial}.')

    #syr pump infuse
    hbpump.set_tvolume(f'{volume} ul') # need to work out somewhere a set volume
    hbpump.infuse()
    time.sleep(2)
    time.sleep(60 * (volume / irate))
    #need to put a sleep to make code wait for withdrawal before next step
    hbpump.clear_volume()

    #for needle to go up down up
    g.bCommand('Z125')
    g.bCommand('Z85')
    g.bCommand('Z125')


g.go_to_home()

volumes_3 = {10:17, 25:18, 50:19, 100:20}

for volume, vial in volumes_3.items():
    #go to the stock sol vial
    g.go_to_vial(24)
    g.bCommand('Z85')
    
    #syr pump withdraw
    hbpump.set_tvolume(f'{volume} ul') # need to work out somewhere a set volume
    hbpump.withdraw()
    time.sleep(2)
    time.sleep(60 * (volume / wrate))
    #need to put a sleep to make code wait for withdrawal before next step
    hbpump.clear_volume()

    #for needle to go up
    g.bCommand('Z125')
    
    #go to vial
    g.go_to_vial(vial)
    g.bCommand('Z85')
    print(f'At vial {vial}.')

    #syr pump infuse
    hbpump.set_tvolume(f'{volume} ul') # need to work out somewhere a set volume
    hbpump.infuse()
    time.sleep(2)
    time.sleep(60 * (volume / irate))
    #need to put a sleep to make code wait for withdrawal before next step
    hbpump.clear_volume()

    #for needle to go up down up
    g.bCommand('Z125')
    g.bCommand('Z85')
    g.bCommand('Z125')

g.go_to_home()

Target volume set to 10 ul.
Pump withdrawing
Target volume reached
Volumes reset to zero.
At vial 1.
Target volume set to 10 ul.
Pump infusing
Target volume reached
Volumes reset to zero.
Target volume set to 25 ul.
Pump withdrawing
Pump withdrawing
Pump withdrawing
Volumes reset to zero.
At vial 2.
Pump withdrawing
Target volume set to 25 ul.
Pump infusing


KeyboardInterrupt: 

In [37]:
hbpump.command('cvolume')
hbpump.command('ctvolume')
hbpump.command('irate 200 ul/min')
hbpump.command('stop')

['', ':']

In [38]:
g.go_to_home()

### serial dilution

In [47]:
# vial stock 1 mL
# vial A - F 0.5 mL MeCN
# serial dilution

hbpump.clear_volume()

vials = [2, 3, 4, 5, 6, 7]
volume = 500

for vial in vials:
    hbpump.set_tvolume(f'{volume} uL')
    g.go_to_vial(vial - 1)
    g.bCommand('Z85')
    hbpump.withdraw()
    time.sleep(60 * (100 / 1000))
    hbpump.clear_volume()
    g.bCommand('Z125')

    hbpump.set_tvolume(f'{volume} uL')
    g.go_to_vial(vial)
    g.bCommand('Z85')
    hbpump.infuse()
    time.sleep(60 * (100 / 1000))
    hbpump.clear_volume()
    g.bCommand('Z125')
    g.bCommand('Z85')
    g.bCommand('Z125')

Volumes reset to zero.
Target volume set to 500 uL.
Pump withdrawing
Pump withdrawing.


KeyboardInterrupt: 

In [67]:
hbpump.command('ctvolume')
hbpump.command('cvolume')
hbpump.command('irun')

Pump infusing
Pump infusing
Pump infusing


['', '>']

In [68]:
hbpump.command('stop')

['', ':']

In [27]:
g.go_to_vial(3)

In [28]:
g.bCommand('Z85')

bytearray(b'\nZ85\r')

In [80]:

g.go_to_home()

In [41]:
hbpump.command('syrm sge 1000 ul')

['', ':']

In [29]:
g.bCommand('H')

bytearray(b'\nH\r')